In [1]:
import networkx as nx

In [2]:
with open("data/day21.txt", encoding="utf-8") as f:
    data = f.read()

In [3]:
codes = [d for d in data.split("\n")]
numpad = [["7", "8", "9"], ["4", "5", "6"], ["1", "2", "3"], ["#", "0", "A"]]
keypad = [["#", "^", "A"], ["<", "v", ">"]]
directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]
numpad_positions = {numpad[i][j]: (i, j) for i in range(4) for j in range(3)}
keypad_positions = {keypad[i][j]: (i, j) for i in range(2) for j in range(3)}

def is_valid(i, j, rows, cols):
    return 0 <= i < rows and 0 <= j < cols

def create_graph(source):
    G = nx.Graph()
    for i, row in enumerate(source):
        for j, elm in enumerate(row):
            if elm == "#":
                continue
            neighbours = [(i+d[0], j+d[1]) for d in directions if is_valid(i+d[0], j+d[1], len(source), len(source[0]))]
            for ni, nj in neighbours:
                if source[ni][nj] != "#":
                    G.add_edge(source[i][j], source[ni][nj])
    return G
   
G_numpad = create_graph(numpad)
G_keypad = create_graph(keypad)

cache_sequences = {}

def best_sequences(start, end, numpad=False):
    if (start, end) in cache_sequences:
        return cache_sequences[(start, end)]
    
    if numpad:
        shortest = nx.all_shortest_paths(G_numpad, start, end)
    else:
        shortest = nx.all_shortest_paths(G_keypad, start, end)
        
    sequences = []
    for path in shortest:
        sequence = ""
        for l, r in zip(path, path[1:]):
            if numpad:
                l_pos, r_pos = numpad_positions[l], numpad_positions[r]
            else:
                l_pos, r_pos = keypad_positions[l], keypad_positions[r]
            vertical, horizontal = r_pos[0] - l_pos[0], r_pos[1] - l_pos[1]
            sequence += "v" if vertical > 0 else "^" if vertical < 0 else ">" if horizontal > 0 else "<"
        sequences.append(sequence + "A")

    cache_sequences[(start, end)] = sequences
    return sequences

# Problem 1

In [4]:
def type_symbol(nb_robot, start, end):
    if nb_robot == 1:
        return best_sequences(start, end, True)
    
    solutions = []
    sequences = type_symbol(nb_robot - 1, start, end)
    for sequence in sequences:
        new_sequences = [""]
        for s, e in zip("A" + sequence, sequence):
            candidates = best_sequences(s, e)
            new_sequences = [ns + c for c in candidates for ns in new_sequences]
        solutions.extend(new_sequences)
        
    best_length = min([len(s) for s in solutions])
    solutions = [s for s in solutions if len(s) == best_length]
    return solutions

def type_code(nb_robot, code):
    instructions = [""]
    for s, e in zip("A" + code, code):
        candidates = type_symbol(nb_robot, s, e)
        instructions = [i + c for c in candidates for i in instructions]
    return instructions

total = 0
for code in codes:
    code_length = len(type_code(3, code)[0])
    total += int(code[:-1]) * code_length
total 

248108

# Problem 2

In [5]:
cache = {}
def code_length(nb_robot, code, numpad=False):
    if nb_robot == 0:
        return len(code)
    if (nb_robot, code) in cache.keys():
        return cache[(nb_robot, code)]
    
    length = 0
    for s, e in zip("A" + code, code):
        combinations = best_sequences(s, e, numpad)
        length += min([code_length(nb_robot-1, c) for c in combinations])
        
    cache[(nb_robot, code)] = length
    return length

total = 0
for code in codes:
    length = code_length(26, code, True)
    total += int(code[:-1]) * length
total

303836969158972